# Founding

- Test with 5 min timeframe 
* CL high volatility 
* SPY better momentum in a day , can try other timeframe 

# TODO
* research low risk product 
* implement stop loss 
* stream price to algo

In [2]:
import pandas as pd

pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data
import pandas
import  numpy as np
from datetime import datetime, timedelta

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)

pd.options.mode.chained_assignment = None


from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.models.tools import HoverTool

from bokeh.models import LinearAxis, Range1d
output_notebook()

from bokeh.palettes import Category20 as palettes
# itertools handles the cycling
import itertools 

colors = itertools.cycle(palettes[20]) 



%matplotlib notebook

Loading BokehJS ...

# Load Min Data

In [6]:
path= "~/Desktop/tmp/SPY.csv"

df=pd.read_csv(path, delimiter=",")

df.head()



,Dates,Open,Close,High,Low
0,11/16/18 22:30,2718.54,2716.39,2718.54,2716.26
1,11/16/18 22:31,2716.53,2718.58,2719.99,2716.53
2,11/16/18 22:32,2718.73,2717.48,2719.59,2717.13
3,11/16/18 22:33,2717.27,2718.60,2719.70,2716.95
4,11/16/18 22:34,2718.39,2716.17,2718.60,2716.17


In [7]:
df.Dates=pd.to_datetime(df.Dates)
df=df.set_index('Dates')

In [8]:
print(df.head())

#df.tz_localize('Hongkong', level=0)
print(df.describe())
df.head()

print(df.index[0].timestamp())


                        Open    Close     High      Low
Dates                                                  
2018-11-16 22:30:00  2718.54  2716.39  2718.54  2716.26
2018-11-16 22:31:00  2716.53  2718.58  2719.99  2716.53
2018-11-16 22:32:00  2718.73  2717.48  2719.59  2717.13
2018-11-16 22:33:00  2717.27  2718.60  2719.70  2716.95
2018-11-16 22:34:00  2718.39  2716.17  2718.60  2716.17
               Open         Close          High           Low
count  53112.000000  53112.000000  53112.000000  53112.000000
mean    2748.717967   2748.720316   2749.245137   2748.195706
std      130.988141    130.987042    130.789325    131.183235
min     2346.830000   2346.930000   2348.450000   2346.580000
25%     2654.525000   2654.487500   2655.420000   2653.640000
50%     2780.590000   2780.620000   2781.030000   2780.205000
75%     2853.440000   2853.432500   2853.860000   2852.937500
max     2953.970000   2953.960000   2954.130000   2953.560000
1542407400.0


In [11]:
# check time continuouty 

start= df.index[0]

end = df.index[-1]

rownum = -1

day=datetime.strptime('2018-11-01 04:59:59', '%Y-%m-%d %H:%M:%S')

tmp = df[day:]
missing_date = day.date()
missing_count =0 
for row in tmp.itertuples():
    rownum += 1
    
    now =  pd.to_datetime(tmp.index[rownum])
    last_time = pd.to_datetime(tmp.index[rownum-1])
    
    if(now <last_time+timedelta(days=1) )& (now > last_time+timedelta(minutes = 1)):
        if (now.hour==6  and last_time.hour == 4):
            #print(" new Day ", now)
            pass
        else:
            print(" check today ", now, now-last_time," last ", last_time )
            if now.date() != missing_date:
                print(" check today ", missing_date, missing_count )
                missing_count =1
                missing_date=now.date()
                
            missing_count +=1
            
    elif( now > last_time+timedelta(days=1)) & (last_time.weekday==5):
        #print(" friday ", now)
        pass
   



 check today  2018-11-20 22:30:00 0 days 17:16:00  last  2018-11-20 05:14:00
 check today  2018-11-01 0
 check today  2018-11-21 22:30:00 0 days 17:16:00  last  2018-11-21 05:14:00
 check today  2018-11-20 2
 check today  2018-11-24 03:29:00 0 days 01:10:00  last  2018-11-24 02:19:00
 check today  2018-11-21 2
 check today  2018-11-27 22:30:00 0 days 17:16:00  last  2018-11-27 05:14:00
 check today  2018-11-24 2
 check today  2018-11-28 22:30:00 0 days 17:16:00  last  2018-11-28 05:14:00
 check today  2018-11-27 2
 check today  2018-11-29 22:30:00 0 days 17:16:00  last  2018-11-29 05:14:00
 check today  2018-11-28 2
 check today  2018-11-30 22:30:00 0 days 17:16:00  last  2018-11-30 05:14:00
 check today  2018-11-29 2
 check today  2018-12-04 22:30:00 0 days 17:16:00  last  2018-12-04 05:14:00
 check today  2018-11-30 2
 check today  2018-12-07 22:30:00 0 days 17:16:00  last  2018-12-07 05:14:00
 check today  2018-12-04 2
 check today  2018-12-11 22:30:00 0 days 17:16:00  last  2018-12

 check today  2019-05-02 21:30:00 0 days 17:16:00  last  2019-05-02 04:14:00
 check today  2019-05-01 2
 check today  2019-05-03 21:30:00 0 days 17:16:00  last  2019-05-03 04:14:00
 check today  2019-05-02 2
 check today  2019-05-07 21:30:00 0 days 17:16:00  last  2019-05-07 04:14:00
 check today  2019-05-03 2
 check today  2019-05-08 21:30:00 0 days 17:16:00  last  2019-05-08 04:14:00
 check today  2019-05-07 2
 check today  2019-05-09 21:30:00 0 days 17:16:00  last  2019-05-09 04:14:00
 check today  2019-05-08 2
 check today  2019-05-10 21:30:00 0 days 17:16:00  last  2019-05-10 04:14:00
 check today  2019-05-09 2
 check today  2019-05-14 21:30:00 0 days 17:16:00  last  2019-05-14 04:14:00
 check today  2019-05-10 2
 check today  2019-05-15 21:30:00 0 days 17:16:00  last  2019-05-15 04:14:00
 check today  2019-05-14 2
 check today  2019-05-16 21:30:00 0 days 17:16:00  last  2019-05-16 04:14:00
 check today  2019-05-15 2
 check today  2019-05-17 21:30:00 0 days 17:16:00  last  2019-05

# generate trading hours re sampler data

In [12]:
day=datetime.strptime('2018-01-01 04:59:59', '%Y-%m-%d %H:%M:%S')
df = df[day:]
resampled_df = df.resample('1min').interpolate(method='linear')
market = "EQ"
#HKG TIME
#futures 
if market =="FT":
    resampled_df = resampled_df[
                    
                        ( (resampled_df.index.weekday >0) & (resampled_df.index.weekday <= 5)  & (resampled_df.index.hour<=4))
                        | ((resampled_df.index.weekday<=4) & (resampled_df.index.hour>=6)) 
                        

                        #| ((resampled_df.index.weekday==6) & (resampled_df.index.hour>=22))
                       ]
elif market == "FX":
    resampled_df =resampled_df[ ((resampled_df.index.weekday <= 4)  & (resampled_df.index.hour<=4))
                        | ((resampled_df.index.weekday<=4) & (resampled_df.index.hour>=5)) 
                        | ((resampled_df.index.weekday==5) & (resampled_df.index.hour<=4))
                       ]
elif market == "EQ":
    print(" Equity")
    resampled_df =resampled_df[ 
        
                        ( (resampled_df.index.weekday >0) &  (resampled_df.index.weekday <= 5)  & (resampled_df.index.hour<=3))
                        | ((resampled_df.index.weekday<=4) & (resampled_df.index.hour>21))
                        | ((resampled_df.index.weekday<=4) & (resampled_df.index.hour==21) &  (resampled_df.index.minute>=30))
                        
                       ]
elif market =="CRYPTO":
    print("Crypto")


resampled_df['hour']=resampled_df.index.hour
resampled_df['minute']=resampled_df.index.minute
resampled_df['weekday']=resampled_df.index.weekday

def getDfOnDay(dataF , tradeDay):
    if tradeDay.weekday() >= 5:
        return None
    
    startTime = tradeDay+ timedelta(days=-1)
    startTime = startTime.replace(hour=22, minute=0,second=0,microsecond=0)
    endTime = tradeDay.replace(hour=20, minute=59,second=59,microsecond=0)
    
    dailyDf = dataF.loc[startTime:endTime]
    
    return dailyDf
    
tradeDay=datetime(2019,5,2)
tradeDateDf = getDfOnDay(resampled_df , tradeDay)
    

 Equity


# plot over view

In [15]:
plot_df = resampled_df


hover = HoverTool()

hover.tooltips = [
    ("x, y", "@x,@y"), 
]
colNameList=['Open','Close','High','Low']
p=figure( x_axis_type='datetime')
coloer=['red','yellow','green','blue','orange']
tf=[5,10,15,30,60]
for i in range(len(colNameList)):
    p.line(x=plot_df.index ,y=plot_df[colNameList[i]], 
           color=next(colors),legend=colNameList[i]
           )
    
p.tools.append(hover)
show(p)

In [14]:
resampled_df= resampled_df[:'2018-10-29 21:31:01']
resampled_df

,Open,Close,High,Low,hour,minute,weekday
Dates,,,,,,,


In [31]:
'''
bid|askInXX: bid|ask limit price
min|maxInXX: low|high in future
buy|sellSpdInXX: min|max - Open price
buy|sellMddInXX: buy|sell order max drag down
buy|sellPosInXX: buy|sell position indicator
cumPosInXX
unrealPnlINXX:
'''
def genMakingDf(stkHistDf,  timeFrame={5:[-5,5],10:[-10,10],15:[-15,15],30:[-30,30],60:[-60,60]}, maxOpen=30):
    stkDf = stkHistDf
    buy_str='buy'
    sell_str = 'sell'
    
    side_dict ={buy_str:0, sell_str:1}
    
    
    for tf, spd_delta in timeFrame.items():
        shift_win= -tf + 1 ##open at t_i's open and  close|high|low at t_(i+tf-1)'s close|high|low
        
        #print( "spd delta ",'bidIn'+str(tf), spd_delta[0], stkDf.head())

        bid= stkDf['bidIn'+str(tf)]= stkDf['Open']+spd_delta[0]
        ask= stkDf['askIn'+str(tf)]= stkDf['Open']+spd_delta[1]
        
        rollMin = stkDf['minIn'+str(tf)]=stkDf.Low.rolling(tf).min().shift(shift_win)
        rollMax = stkDf['maxIn'+str(tf)]=stkDf.High.rolling(tf).max().shift(shift_win)
        
        
        
        for side,index in side_dict.items():
            
            
            limit_price = stkDf['Open']+spd_delta[index]
            
            if side =='buy':
                stkDf[side+'SpdIn'+str(tf)]= rollMin - stkDf['Open'] 
                tmp = rollMin - limit_price
                tmp[tmp>0]=0
                stkDf[side+'MddIn'+str(tf)]=tmp
                
                
            else:
                stkDf[side+'SpdIn'+str(tf)]= rollMax - stkDf['Open'] 
                tmp = limit_price- rollMax
                tmp[tmp>0]=0
                stkDf[side+'MddIn'+str(tf)]=tmp
                
            
            tmp= stkDf[side+'MddIn'+str(tf)].copy()
            tmp[tmp<0]=1 if side=='buy' else -1
            stkDf[side+'PosIn'+str(tf)]=tmp
            
        cumUsd= (stkDf[buy_str+'PosIn'+str(tf)] * bid +stkDf[sell_str+'PosIn'+str(tf)] * ask ).cumsum()
        stkDf['cumUsdIn'+str(tf)]=cumUsd
        
        stkDf['cumPosIn'+str(tf)]= (stkDf[buy_str+'PosIn'+str(tf)]  +stkDf[sell_str+'PosIn'+str(tf)] ).cumsum()
        
        
        closeRefPrice = stkDf.Close.shift(shift_win) 
        stkDf['unrealPnlIn'+str(tf)] =  stkDf['cumPosIn'+str(tf)]*closeRefPrice - stkDf[ 'cumUsdIn'+str(tf)]  #open at t_i's open and  close at t_(i+tf-1)'s close 
        
        
        
    ## print info
    print(" trade count buy5 ",stkDf.buyPosIn5.abs().sum(),", sell5 ",stkDf.sellPosIn5.abs().sum())
    
    '''
    test datafram calculation
    
    side_str= buy_str
    bid=stkDf['Open']+timeFrame[5][0]
    ask = stkDf['Open']+timeFrame[5][1]
    tmp= (stkDf[buy_str+'PosIn'+str(5)] * bid +stkDf[sell_str+'PosIn'+str(5)] * ask ).cumsum()
    tmp_pos= (stkDf[buy_str+'PosIn'+str(5)]  +stkDf[sell_str+'PosIn'+str(5)] ).cumsum()
    stkDf['cumUsd'+str(5)]=tmp
    stkDf['cumPos']=tmp_pos
    
    #unreal pnl 
    
    stkDf['unrealPnl'] =  stkDf['cumPos']*stkDf.Close.shift(-5+1) - stkDf[ 'cumUsd'+str(5)]  #open at t_i's open and  close at t_(i+tf-1)'s close 
    
    
        
    tmp=bid-stkDf.Low.rolling(5).min()
    
    stkDf['max']=stkDf.High.rolling(5).max()
    stkDf['max-shf']=stkDf.High.rolling(5).max().shift(-5+1)   #open at t_i's open and  close at t_(i+tf-1)'s close 
    stkDf['ask']=stkDf['Open']+timeFrame[5][1]
    
    print(stkDf[['Open','High','max', 'max-shf','ask','Close','sellNLossIn5','buyPosIn5','sellPosIn5','cumPosIn5','cumUsdIn5','unrealPnlIn5']].head(20))
    '''
    
    return stkDf

def getHourlyStats(df, timeFrame={5:[-2,2],10:[-10,10],15:[-15,15],30:[-30,30],60:[-60,60]}):
    df = genMakingDf(df,timeFrame)
    colNameList = []
    for tf, val in timeFrame.items():
        colNameList.append('buySpdIn'+str(tf))
        colNameList.append('sellSpdIn'+str(tf))
    
    spd_stats = df[colNameList].groupby(pd.Grouper(freq='h')).mean()
    return spd_stats

       


# Plot Spread 

* hourly mean of one day

In [372]:
from bokeh.palettes import Category20 as palettes
# itertools handles the cycling
import itertools 

colors = itertools.cycle(palettes[20]) 

tradeOnDateDf = getDfOnDay(resampled_df , datetime(2019,5,7))
hour_stats = getHourlyStats(tradeOnDateDf )
hour_stats

timeFrame={5:[-2,2],10:[-10,10],15:[-15,15],30:[-30,30],60:[-60,60]}
colNameList = []
for tf, val in timeFrame.items():
    colNameList.append('buySpdIn'+str(tf))
    colNameList.append('sellSpdIn'+str(tf))

    

hover = HoverTool()

hover.tooltips = [
    ("Index", "@index"), 
]

p=figure( x_axis_type='datetime')
coloer=['red','yellow','green','blue','orange']
tf=[5,10,15,30,60]
for i in range(len(colNameList)):
    p.line(x=hour_stats.index ,y=hour_stats[colNameList[i]], 
           color=next(colors),legend=colNameList[i]
           )
    
p.tools.append(hover)
show(p)



 trade count buy5  0.0 , sell5  0.0


In [32]:
start_date=datetime(2019,1,1)
end_date = datetime(2019,3,1)


'''
 {22 :{buySpdIn5:[] , sellSpdIn5:[].... } 
        
 

'''
hour_stats_dict = {}

for tradeDay in pd.date_range(start_date, end_date):
    
    tradeOnDateDf = getDfOnDay(resampled_df , tradeDay)
    
    if tradeOnDateDf is None:
        continue
        
    #print( tradeOnDateDf[0]) 
    #print( tradeDay)
    hour_stats = getHourlyStats(tradeOnDateDf)
    hour_stats['hour']= hour_stats.index.hour
    #print(hour_stats.buySpdIn5,hour_stats.columns)
    
    #iter hours 
    for index, row in hour_stats.iterrows():
        hour = row['hour']
        
        if hour not in hour_stats_dict:
            hour_stats_dict[hour]={}

        for column in hour_stats.columns:
            if column=='hour':
                continue
            if column not in hour_stats_dict[hour]:
                hour_stats_dict[hour][column]=[]
            
            hour_stats_dict[hour][column].append(row[column])
    
hour_params ={} # col: ( hr, mean)
for hr, val in hour_stats_dict.items():
    for col, arr in val.items():
        mean= sum(arr)/len(arr)
        if col not in hour_params:
            hour_params[col]={}
        if hr not in hour_params[col]:
            hour_params[col][hr]={}
        hour_params[col][hr]=mean
        
print("buySpdIn5: ",hour_params['buySpdIn5'])

print("sellSpdIn5: ",hour_params['sellSpdIn5'])



 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sell5  0.0
 trade count buy5  0.0 , sel

KeyError: 'buySpdIn5'

# use mean or median in param?

In [39]:
TIMEFRAME=5
timeFrame={1:[-1,1],5:[-2,2],10:[-10,10],15:[-15,15],30:[-30,30],60:[-60,60]}

#filter by hour 
train_df = resampled_df['2019-01-01':'2019-03-1']

paramsDict ={}
for hour in range(0,24):
    
    tmp = genMakingDf(train_df,timeFrame)
    tmp['hour']=tmp.index.hour
    same_hour_data = tmp[tmp.hour==hour]
    
    

    hover = HoverTool()

    hover.tooltips = [
        ("(x,y)", "(@x, @y)"), 
    ]

    p=figure( title=' hour %d dist'%(hour), x_axis_type='datetime')
    coloer=['red','yellow','green','blue','orange']
    tf=[5,10,15,30,60]

    #for i in range(len(colNameList)):
    for i in range(2):
        p.circle(x=same_hour_data.index ,y=same_hour_data[colNameList[i]], 
               color=next(colors),legend=colNameList[i],
               )

    p.tools.append(hover)
    
    print("hour %d median "%hour, same_hour_data['buySpdIn'+str(TIMEFRAME)].median(), same_hour_data['sellSpdIn'+str(TIMEFRAME)].median())
    #paramsDict[hour]=(same_hour_data.buySpdIn5.median(),same_hour_data.sellSpdIn5.median())
    
    print("hour %d mean "%hour, same_hour_data['buySpdIn'+str(TIMEFRAME)].mean(), same_hour_data['sellSpdIn'+str(TIMEFRAME)].mean())
    paramsDict[hour]=(same_hour_data['buySpdIn'+str(TIMEFRAME)].mean(),same_hour_data['sellSpdIn'+str(TIMEFRAME)].mean())
    #show(p)
print(paramsDict)

same_hour_data['buySpdIn'+str(TIMEFRAME)].mean()

 trade count buy5  4316.0 , sell5  3816.0
hour 0 median  -0.9899999999997817 0.9900000000000091
hour 0 mean  -1.3433279655489667 1.3396212121212172
 trade count buy5  4316.0 , sell5  3816.0
hour 1 median  -0.819999999999709 0.7100000000000364
hour 1 mean  -1.1579123553045267 1.004174242424253
 trade count buy5  4316.0 , sell5  3816.0
hour 2 median  -0.6700000000000728 0.5900000000001455
hour 2 mean  -1.084523260211613 0.9151325757575748
 trade count buy5  4316.0 , sell5  3816.0
hour 3 median  -0.930000000000291 0.6700000000000728
hour 3 mean  -1.623027593908169 1.5037569415264134
 trade count buy5  4316.0 , sell5  3816.0
hour 4 median  nan nan
hour 4 mean  nan nan
 trade count buy5  4316.0 , sell5  3816.0
hour 5 median  nan nan
hour 5 mean  nan nan
 trade count buy5  4316.0 , sell5  3816.0
hour 6 median  nan nan
hour 6 mean  nan nan
 trade count buy5  4316.0 , sell5  3816.0
hour 7 median  nan nan
hour 7 mean  nan nan
 trade count buy5  4316.0 , sell5  3816.0
hour 8 median  nan nan
hour

-1.6790830956352667

In [791]:
def genPramaPerHour(testDf,timeframe=5, hourParamDict=None):
    if hourParamDict is None:
        hourParamDict={}
        for hr in range(0,24):
            hourParamDict[hr]=(-0.05,0.05)
        
    
    '''
    init testDf
    adding column 
    bidIn5, askIn5
    
    '''
    stkDf = testDf.copy()
    stkDf[ 'bidIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    stkDf[ 'askIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    
    stkDf[ 'minIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    stkDf[ 'maxIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    
    stkDf[ 'buyMddIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    stkDf[ 'sellMddIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    
    stkDf[ 'buyPosIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    stkDf[ 'sellPosIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    
    stkDf[ 'cumUsdIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    stkDf[ 'cumPosIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    stkDf[ 'unrealPnlIn'+str(timeframe)]  =  pd.Series(None, index=stkDf.index)
    
    #print(stkDf.head(10))
        
    
    buy_str='buy'
    sell_str = 'sell'
    
    side_dict ={buy_str:0, sell_str:1}
    
    shift_win= -timeframe + 1
    rollMin = stkDf['minIn'+str(timeframe)]=stkDf.Low.rolling(timeframe).min().shift(shift_win)
    rollMax = stkDf['maxIn'+str(timeframe)]=stkDf.High.rolling(timeframe).max().shift(shift_win)
    
    for hr, val in hourParamDict.items():
        bid = stkDf.loc[ stkDf.index.hour==hr,['bidIn'+str(timeframe)] ] = stkDf['Open']+ hourParamDict[hr][0]
        ask = stkDf.loc[ stkDf.index.hour==hr,['askIn'+str(timeframe)] ] = stkDf['Open']+ hourParamDict[hr][1]
            
    for row in stkDf.itertuples():
        #print( row)
        #hr = row.hour  
        for side,val in side_dict.items():
            if side =='buy':
                tmp = stkDf.at[row.Index, 'minIn'+str(timeframe)] - stkDf.at[row.Index, 'bidIn'+str(timeframe)]
                
                stkDf.at[row.Index, 'buyMddIn'+str(timeframe)] = tmp if tmp<0 else 0 
                
                stkDf.at[row.Index, 'buyPosIn'+str(timeframe)] = 1 if tmp<0 else 0
                
                
            else:
                tmp = stkDf.at[row.Index, 'askIn'+str(timeframe)]- stkDf.at[row.Index, 'maxIn'+str(timeframe)]
                stkDf.at[row.Index, 'sellMddIn'+str(timeframe)] = tmp if tmp<0 else 0
                stkDf.at[row.Index, 'sellPosIn'+str(timeframe)] = -1 if tmp<0 else 0

    
    cumUsd= (stkDf[buy_str+'PosIn'+str(timeframe)] * bid +stkDf[sell_str+'PosIn'+str(timeframe)] * ask ).cumsum()
    stkDf['cumUsdIn'+str(timeframe)]=cumUsd

    stkDf['cumPosIn'+str(timeframe)]= (stkDf[buy_str+'PosIn'+str(timeframe)]  +stkDf[sell_str+'PosIn'+str(timeframe)] ).cumsum()


    closeRefPrice = stkDf.Close.shift(shift_win) 
    stkDf['unrealPnlIn'+str(timeframe)] =  stkDf['cumPosIn'+str(timeframe)]*closeRefPrice - stkDf[ 'cumUsdIn'+str(timeframe)]  #open at t_i's open and  close at t_(i+tf-1)'s close 
    
    print(" counts ", abs(stkDf['buyPosIn'+str(timeframe)].sum())+ abs(stkDf['sellPosIn'+str(timeframe)].sum()))
    
    return stkDf
testDf = resampled_df['2018-09-01 00:0:00':'2018-09-2 0:00:00'][['Open','High','Low','Close','hour']]

'''
hourParamDict={}
for hr in range(0,24):
    hourParamDict[hr]=(-0.0002,0.0001)
'''
    
bt_result = genPramaPerHour(testDf, timeframe=TIMEFRAME, hourParamDict=paramsDict)

 counts  389.0


# Plot Backtest Result

In [792]:
#tmp = resampled_df['2019-05-16':]

#tmp.loc[ tmp.index.hour==0,['bidIn5'] ] = tmp['Open']*0
#tmp[['Open','bidIn5']]

#tmp[['bidIn5','sellIn5','buyPosIn5','sellPosIn5']].head(10)



colNameList = ['Open','Close']
for tf, val in timeFrame.items():
    colNameList.append('buySpdIn'+str(tf))
    colNameList.append('sellSpdIn'+str(tf))
    

tf = TIMEFRAME

'''
bt_result = bt_result[['Open','Close','minIn'+str(5),
                       'bidIn5','buyMddIn5',
                       'buyPosIn5','askIn5',
                       'sellMddIn5','sellPosIn5',
                       'cumUsdIn5','cumPosIn5','unrealPnlIn5']]
                       '''


hover = HoverTool()

hover.tooltips = [
    ("x,y", "@x,@y"), 
]

p=figure( x_axis_type='datetime')
coloer=['red','yellow','green','blue','orange']

for colName in ['unrealPnlIn'+str(tf)]:
    p.line(x=bt_result.index ,y=bt_result[colName], 
           color=next(colors),legend=colName
           )
    
p.tools.append(hover)
show(p)


p=figure( x_axis_type='datetime', y_range=(bt_result['Close'].min(), bt_result['Close'].max()))

#p.extra_y_ranges = {"Close": Range1d(start=bt_result['Close'].min(), end=bt_result['Close'].max())}
#p.add_layout(LinearAxis(y_range_name="Close"), 'left')
p.line(x=bt_result.index ,y=bt_result['Close'], 
       color=next(colors),legend='Close'
       #y_range_name='Close'
       )

#p.y_range = Range1d(0, .05)
p.extra_y_ranges = {"cumPos": Range1d(start=bt_result['cumPosIn'+str(tf)].min(), end=bt_result['cumPosIn'+str(tf)].max())}
p.add_layout(LinearAxis(y_range_name="cumPos"), 'right')

p.line(x=bt_result.index ,y=bt_result['cumPosIn'+str(tf)], 
       color=next(colors),legend='cumPosIn'+str(tf)
       ,y_range_name='cumPos'
       )
    
p.tools.append(hover)
show(p)

In [752]:
'''
bt_result[['Open','Close',
                       'bidIn5','buyMddIn5',
                       'buyPosIn5','askIn5',
                       'sellMddIn5','sellPosIn5',
                       'cumUsdIn5','cumPosIn5','unrealPnlIn5']]
'''
#testDf
bt_result

#resampled_df

,Open,High,Low,Close,hour,bidIn15,askIn15,minIn15,maxIn15,buyMddIn15,sellMddIn15,buyPosIn15,sellPosIn15,cumUsdIn15,cumPosIn15,unrealPnlIn15
Dates,,,,,,,,,,,,,,,,
2018-06-17 00:00:00,6486.86,6491.42,6486.86,6491.42,0,6485.040,6488.41,6486.86,6504.62,0.000,-16.21,0.0,-1.0,-6488.520,-1.0,-13.320
2018-06-17 00:01:00,6491.42,6491.42,6491.42,6491.42,0,6489.600,6492.97,6487.13,6506.84,-2.470,-13.87,1.0,-1.0,-6490.985,-1.0,-15.855
2018-06-17 00:02:00,6491.42,6491.42,6491.42,6491.42,0,6489.600,6492.97,6487.13,6506.84,-2.470,-13.87,1.0,-1.0,-6493.450,-1.0,-13.390
2018-06-17 00:03:00,6491.42,6491.42,6488.89,6488.89,0,6489.600,6492.97,6487.13,6511.14,-2.470,-18.17,1.0,-1.0,-6495.915,-1.0,-13.615
2018-06-17 00:04:00,6488.89,6488.89,6487.13,6487.13,0,6487.070,6490.44,6487.13,6511.14,0.000,-20.70,0.0,-1.0,-12986.465,-2.0,-32.595
2018-06-17 00:05:00,6487.13,6487.13,6487.13,6487.13,0,6485.310,6488.68,6487.13,6511.14,0.000,-22.46,0.0,-1.0,-19475.255,-3.0,-53.335
2018-06-17 00:06:00,6487.13,6487.13,6487.13,6487.13,0,6485.310,6488.68,6487.13,6511.14,0.000,-22.46,0.0,-1.0,-25964.045,-4.0,-74.075
2018-06-17 00:07:00,6487.13,6487.13,6487.13,6487.13,0,6485.310,6488.68,6487.13,6511.14,0.000,-22.46,0.0,-1.0,-32452.835,-5.0,-94.815
2018-06-17 00:08:00,6487.13,6487.13,6487.13,6487.13,0,6485.310,6488.68,6487.13,6511.14,0.000,-22.46,0.0,-1.0,-38941.625,-6.0,-115.555
